In [1]:
print("Ok")

Ok


In [2]:
import os
from dotenv import load_dotenv


In [5]:
#to access the code 
load_dotenv()
OPENAI_API_KEY = os.environ.get('OPENAI_API_KEY')
 

In [7]:
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [10]:
from langchain.document_loaders import PyPDFLoader

In [12]:
%pwd

'/home/akv/Documents/LM-Works/lm-works-ashok/research'

In [13]:
%cd ..
%pwd

/home/akv/Documents/LM-Works/lm-works-ashok


'/home/akv/Documents/LM-Works/lm-works-ashok'

In [17]:

file_path = "data/Earnings-Jan13-2025_1.pdf"
# load the data use the langchain pdf loader 
loader = PyPDFLoader(file_path)
data = loader.load()
len(data)

4

In [ ]:
qa_gen = ""
# extract the content of the 4 pages 
for page in data:
    qa_gen = qa_gen + page.page_content

print(qa_gen)

In [20]:
#chunks
from langchain.text_splitter import TokenTextSplitter 
splitter_for_qna = TokenTextSplitter(
    model_name= "gpt-3.5-turbo",
    chunk_size=10000,
    chunk_overlap = 200
)    
# in above we are using TokenTextSplitter , In this case  model  will decide in the above for chunk_size


In [21]:
chunk_ques_gen = splitter_for_qna.split_text(qa_gen)

In [23]:
chunk_ques_gen


['Page 1 of 6 \n \n \n \n \n \n \n \n \nUnitedHealth Group Reports 2024 Results \n \n \n• Revenues of $400.3 Billion Grew 8% Year-Over-Year  \n• Domestic Consumers Served by UnitedHealthcare Grew 2.1 Million  \n• Value-Based Care Patients Served by Optum Grew 600,000 \n• Cash Flows from Operations were $24.2 Billion or 1.6x Net Income  \n• Full Year, Fourth Quarter Net Earnings were $15.51, $5.98 Per Share  \n• Full Year, Fourth Quarter Adjusted Net Earnings were $27.66, $6.81 Per Share \n• Company Affirms 2025 Performance Outlook \n \n \n(January 16, 2025) UnitedHealth Group (NYSE: UNH) reported full year and fourth quarter 2024 \nresults reflecting diversified growth in serving people more extensively at Optum and \nUnitedHealthcare.  \n \n“The people of UnitedHealth Group remain focused on making high-quality, affordable health care \nmore available to more people while making the health system easier to navigate for patients and \nproviders, positioning us well for growth in 2025,”

In [25]:
len(chunk_ques_gen)

1

In [29]:
type(chunk_ques_gen)


list

In [28]:
 
type(chunk_ques_gen[0])

str

In [30]:
from langchain.docstore.document import Document 


In [31]:
# we can't pass str to LLM  , so we need to pass the str to document format using above library , As emebedding model understand Document format

document_qna_gen = [Document(page_content= txt) for txt in chunk_ques_gen]
#list comprehension used to convert  in above 

In [32]:
document_qna_gen 

[Document(metadata={}, page_content='Page 1 of 6 \n \n \n \n \n \n \n \n \nUnitedHealth Group Reports 2024 Results \n \n \n• Revenues of $400.3 Billion Grew 8% Year-Over-Year  \n• Domestic Consumers Served by UnitedHealthcare Grew 2.1 Million  \n• Value-Based Care Patients Served by Optum Grew 600,000 \n• Cash Flows from Operations were $24.2 Billion or 1.6x Net Income  \n• Full Year, Fourth Quarter Net Earnings were $15.51, $5.98 Per Share  \n• Full Year, Fourth Quarter Adjusted Net Earnings were $27.66, $6.81 Per Share \n• Company Affirms 2025 Performance Outlook \n \n \n(January 16, 2025) UnitedHealth Group (NYSE: UNH) reported full year and fourth quarter 2024 \nresults reflecting diversified growth in serving people more extensively at Optum and \nUnitedHealthcare.  \n \n“The people of UnitedHealth Group remain focused on making high-quality, affordable health care \nmore available to more people while making the health system easier to navigate for patients and \nproviders, posit

In [34]:
type(document_qna_gen[0])

langchain_core.documents.base.Document

In [42]:
#chunks - again why ?
#two-step chunking (or hierarchical chunking) is sometimes necessary to ensure that the resulting 
# chunks preserve context and semantic meaning while adhering to token limits.
from langchain.text_splitter import TokenTextSplitter 
splitter_ans_qna = TokenTextSplitter(
    model_name= "gpt-3.5-turbo",
    chunk_size=1000,
    chunk_overlap = 200
)    

In [43]:
document_ans_qna = splitter_ans_qna.split_documents(document_qna_gen)

In [44]:
document_ans_qna

[Document(metadata={}, page_content='Page 1 of 6 \n \n \n \n \n \n \n \n \nUnitedHealth Group Reports 2024 Results \n \n \n• Revenues of $400.3 Billion Grew 8% Year-Over-Year  \n• Domestic Consumers Served by UnitedHealthcare Grew 2.1 Million  \n• Value-Based Care Patients Served by Optum Grew 600,000 \n• Cash Flows from Operations were $24.2 Billion or 1.6x Net Income  \n• Full Year, Fourth Quarter Net Earnings were $15.51, $5.98 Per Share  \n• Full Year, Fourth Quarter Adjusted Net Earnings were $27.66, $6.81 Per Share \n• Company Affirms 2025 Performance Outlook \n \n \n(January 16, 2025) UnitedHealth Group (NYSE: UNH) reported full year and fourth quarter 2024 \nresults reflecting diversified growth in serving people more extensively at Optum and \nUnitedHealthcare.  \n \n“The people of UnitedHealth Group remain focused on making high-quality, affordable health care \nmore available to more people while making the health system easier to navigate for patients and \nproviders, posit

In [45]:
len(document_ans_qna)

3